## 1. Create endpoint config
In this section we will describe how we can create a model endpoint.
If you already have endpoint , please go to last cell for invocation, give the correct end point name and start inferences.

For generating sagemaker end point we will need to generate endpoint config. So generating endpoint config first

In [1]:
%store -r # you can read model from another notebook via this method
print(v_model_name)

ds-mlops-linear-learner


In [8]:
import sys
import math
import boto3
import sagemaker
from time import gmtime, strftime
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)


In [9]:
# Instance config for endpoint
endpoint_config_name = 'endpoint-config-71726f920c2011ecbb75028939f4d999'
v_model_name = "ds-mlops-xgboost" # To be changed
Instance_Type = "ml.m5.large"
VolumeSizeIn_GB = 20
Instance_Count = 1
endpoint_name = 'endpoint-e1191a86f78411eb8b02028939f4d999' # name of model endpoint
datacaploc='s3://ds-mlops-s3/data/capture/'


**Note :** We will use able endpoint config name to cretae model endpoint

## 2. Delete endpoint config

Incase if you want to delete endpoint config and recreate it we can use below code snippet

In [10]:
response = client.delete_endpoint_config(
    EndpointConfigName=endpoint_config_name
)

In [11]:
# Creating endpoint config
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": Instance_Type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": v_model_name,
            "VariantName": "AllTraffic",
        }
    ],
    DataCaptureConfig={
        'EnableCapture': True,
        'InitialSamplingPercentage': 100,
        'DestinationS3Uri': datacaploc,
        'CaptureOptions': [
            {
                'CaptureMode': 'Input'
            },
             {
                'CaptureMode': 'Output'
            },
        ],
        'CaptureContentTypeHeader': {
            'CsvContentTypes': [
                'text/csv',
            ],
            'JsonContentTypes': [
                'application/json',
            ]
        }        
    }
)

print(f"Endpoint Config Arn: {create_endpoint_config_response['EndpointConfigArn']}")
      

endpoint-config-71726f920c2011ecbb75028939f4d999
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:014257795134:endpoint-config/endpoint-config-71726f920c2011ecbb75028939f4d999


## 3. Delete endpoint
Incase if you want to delete endpoint  and recreate it we can use below code snippet

In [12]:
client.delete_endpoint(EndpointName=endpoint_name)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-east-1:014257795134:endpoint/endpoint-e1191a86f78411eb8b02028939f4d999".

## 4.Create Endpoint

In this notebook we create endpoint using endpoint which we have created

In [13]:
# Importing required library
import sys
import math
import boto3
import sagemaker
import time    

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)


In [14]:
# Creating model endpoint
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
while status == "Creating":
    print(f"Status: {status}")
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]

print(f"Arn: {resp['EndpointArn']}")
print(f"Status: {status}")

endpoint-e1191a86f78411eb8b02028939f4d999
arn:aws:sagemaker:us-east-1:014257795134:endpoint/endpoint-e1191a86f78411eb8b02028939f4d999
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:014257795134:endpoint/endpoint-e1191a86f78411eb8b02028939f4d999
Status: InService


## 5.Inference

In this section we will validate scoring logics. In below code snippet we are scoring with 14 predictor values and finding the result

In [15]:
payload={'year': 2018, 'manufacturer': 5, 'model': '147', 'condition': '2', 'cylinders': '4', 'fuel': '2', 'odometer': '2000', 'title_status': 0, 'transmission': '2', 'drive': '3', 'type': '8', 'paint_color': '6'}
scoring_vector = [payload['year'],payload['manufacturer'],payload['model'],payload['condition'],payload['cylinders'],payload['fuel'],payload['odometer'],payload['title_status'],payload['transmission'],payload['drive'],payload['type'],payload['paint_color']]
xgbody = ','.join([str(item) for item in scoring_vector])
client = boto3.client('sagemaker-runtime')
response_model_xgboost = client.invoke_endpoint(EndpointName=endpoint_name,
                               ContentType='text/csv',
                               Body=xgbody)
result_xgboost = float(response_model_xgboost["Body"].read())
print(result_xgboost)

7748.38232421875


**Note:** Similarly you can change the test vector and can see how endpoint is working

## Clean-up steps

https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html

In [ ]:
# Clean up end point
# client = boto3.client("sagemaker", region_name=region)
# response=client.delete_endpoint(EndpointName=endpoint_name)